In [1]:
import logging
from collections import namedtuple
import random
from numpy.random import choice
import functools
import copy

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __setRows__(self, rows):
        self._rows=copy.deepcopy(rows)

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [3]:
class MinMaxAgent:
    def __init__(self,num_rows):
        self._num_rows_=num_rows
        self._analyzed_moves_=dict()
        self._rules=dict()


    def pickmove(self,nim):
        if tuple(nim._rows) in self._analyzed_moves_:
            return self._analyzed_moves_[tuple(nim._rows)][0]
        moves=self.__possiblemoves(tuple(nim._rows))
        evals=[]
        for move in moves:
            nimCopy=Nim(self._num_rows_)
            nimCopy.__setRows__(nim._rows)
            #print(f"Move to ev {move} while nim rows are {nimCopy._rows} and level is {0}")
            nimCopy.nimming(move)
            evals.append(self.minmax(nimCopy,1,False))
        max,maxIndex=None,None
        #logging.debug(f"PICKMOVE Finished evaluations for state {nim} and myturn {True} with moves {moves} and evals {evals}")
        for _ in range(len(evals)):
            if max is None or evals[_]>max:
                max=evals[_]
                maxIndex=_
        return moves[maxIndex]


    def minmax(self,nim,val,myTurn):
        if not nim:
            return -1*val if myTurn else int(100/val)
        else:
            if myTurn and tuple(nim._rows) in self._analyzed_moves_:
                return self._analyzed_moves_[tuple(nim._rows)][1]
            moves=self.__possiblemoves(tuple(nim._rows))
            evals=[]
            for move in moves:
                nimCopy=Nim(self._num_rows_)
                nimCopy.__setRows__(nim._rows)
                #print(f"Move to ev {move} while nim rows are {nimCopy._rows} and level is {val}")
                nimCopy.nimming(move)
                evals.append(self.minmax(nimCopy,val+1,not myTurn))
            #print(f"Finished evaluating level {val}")
            #logging.debug(f"Finished evaluations for state {nim} and myturn {myTurn} with moves {moves} and evals {evals}")
            if myTurn:
                max,maxIndex=None,None
                for _ in range(len(evals)):
                    if max is None or evals[_]>max:
                        max=evals[_]
                        maxIndex=_
                self._analyzed_moves_[tuple(nim._rows)]=(moves[maxIndex],max)
                return max
            else:
                return min(evals)

                

    def __possiblemoves(self,state):
        #moves=[(row,toTake,0,0) for row in range(self._num_rows_) for toTake in range(state[row])]
        #logging.debug(f"Moves for state {state} have len{len(moves)}")
        #return moves
        return [(row,toTake+1) for row in range(self._num_rows_) for toTake in range(state[row])]
        

In [4]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def _nimsum(state):
    return functools.reduce(lambda a,b : a^b,state)

def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def optimal(state: Nim) -> Nimply:
    nimsum=_nimsum(state._rows)
    #logging.debug(f"For state {state} nimsum is {nimsum}")
    if nimsum==0:
        return pure_random(state)
    else:
        for _ in reversed(range(len(state._rows))):
            #logging.debug(f"Nimsum between {state._rows[_]} and {nimsum} is {state._rows[_]^nimsum}")
            if state._rows[_]^nimsum<state._rows[_]:
                return (_,state._rows[_]-(state._rows[_]^nimsum))
        return pure_random(state)

In [5]:
wins=0
logging.getLogger().setLevel(logging.DEBUG)
myPlayer=MinMaxAgent(11)
starting=True#random.choice([True,False])
player=0
NUM_GAMES=1
for _ in range(NUM_GAMES):
    nim=Nim(11)
    logging.debug(f"In this game I'm player #{0 if starting else 1} and nim is {nim} with nimsum {_nimsum(nim._rows)}")
    while nim:
        if starting!=player:
            ply=myPlayer.pickmove(nim)
        else:
            ply=optimal(nim)
        nim.nimming(ply)
        logging.debug(f"After player {player} move now rows are {nim} with nimsum {_nimsum(nim._rows)}")
        player=1-player
    winner=1-player
    won=(winner==0 and starting) or (winner==1 and not starting)
    if won:
        wins+=1
    starting=True#random.choice([True,False])

logging.debug(f"After {NUM_GAMES} my player won {wins} games!")

DEBUG:root:In this game I'm player #0 and nim is <1 3 5 7 9 11 13 15 17 19 21> with nimsum 23


KeyboardInterrupt: 